In [1]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit  # 自动初始化 CUDA 驱动

# 日志记录器
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

# 指定显式批处理模式
EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

def build_engine(onnx_file_path, engine_file_path, max_workspace_size=1<<30):
    with trt.Builder(TRT_LOGGER) as builder, \
         builder.create_network(EXPLICIT_BATCH) as network, \
         trt.OnnxParser(network, TRT_LOGGER) as parser:
        
        builder.max_workspace_size = max_workspace_size  # 分配最大工作空间大小

        # 读取 ONNX 模型文件
        with open(onnx_file_path, "rb") as model:
            if not parser.parse(model.read()):
                print("解析 ONNX 模型失败:")
                for error in range(parser.num_errors):
                    print(parser.get_error(error))
                return None

        # 构建 engine
        engine = builder.build_cuda_engine(network)
        if engine is None:
            print("构建 engine 失败")
            return None

        # 序列化 engine 到文件
        with open(engine_file_path, "wb") as f:
            f.write(engine.serialize())
        print("Engine 已保存到:", engine_file_path)
        return engine

if __name__ == "__main__":
    onnx_path = "/home/bydguikong/yy_ws/PlanScope/onnx/model.onnx"       # ONNX 模型路径
    engine_path = "/home/bydguikong/yy_ws/PlanScope/onnx/model.engine"   # 输出的 TensorRT engine 文件路径
    build_engine(onnx_path, engine_path)


ModuleNotFoundError: No module named 'tensorrt'